In [1]:
pip install pandas pyarrow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [4]:
df = pd.read_parquet("src/nyc_data_crime.parquet")
print(df.head())

  cad_evnt_id              create_date            incident_date incident_time  \
0    99844077  2024-01-01T00:00:00.000  2024-01-01T00:00:00.000      00:01:13   
1    99844080  2024-01-01T00:00:00.000  2024-01-01T00:00:00.000      00:01:21   
2    99844104  2024-01-01T00:00:00.000  2024-01-01T00:00:00.000      00:03:04   
3    99844141  2024-01-01T00:00:00.000  2024-01-01T00:00:00.000      00:05:51   
4    99844289  2024-01-01T00:00:00.000  2024-01-01T00:00:00.000      00:14:21   

  nypd_pct_cd    boro_nm             patrl_boro_nm geo_cd_x geo_cd_y  \
0           6  MANHATTAN     PATROL BORO MAN SOUTH   981882   209075   
1         103     QUEENS  PATROL BORO QUEENS SOUTH  1042321   195155   
2          44      BRONX         PATROL BORO BRONX  1004911   240882   
3          70   BROOKLYN   PATROL BORO BKLYN SOUTH   994965   166160   
4          24  MANHATTAN     PATROL BORO MAN NORTH   992586   227883   

  radio_code                                        typ_desc      cip_jobs  \
0 

In [21]:
print(sorted(df["typ_desc"].unique().tolist()))
#181 unique values
df["typ_desc"].nunique()
df["cip_jobs"].unique()


['ASSAULT (IN PROGRESS): KNIFE/CHILD ABUSE', 'ASSAULT (IN PROGRESS): KNIFE/FAMILY', 'ASSAULT (IN PROGRESS): KNIFE/INSIDE', 'ASSAULT (IN PROGRESS): KNIFE/LTD ACC HWY', 'ASSAULT (IN PROGRESS): KNIFE/OUTSIDE', 'ASSAULT (IN PROGRESS): KNIFE/SCHOOL', 'ASSAULT (IN PROGRESS): KNIFE/TRANSIT', 'ASSAULT (IN PROGRESS): OTHER/CHILD ABUSE', 'ASSAULT (IN PROGRESS): OTHER/FAMILY', 'ASSAULT (IN PROGRESS): OTHER/INSIDE', 'ASSAULT (IN PROGRESS): OTHER/LTD ACC HWY', 'ASSAULT (IN PROGRESS): OTHER/OUTSIDE', 'ASSAULT (IN PROGRESS): OTHER/SCHOOL', 'ASSAULT (IN PROGRESS): OTHER/TRANSIT', 'ASSAULT (IN PROGRESS): SHOTS/CHILD', 'ASSAULT (IN PROGRESS): SHOTS/FAMILY', 'ASSAULT (IN PROGRESS): SHOTS/INSIDE', 'ASSAULT (IN PROGRESS): SHOTS/LTD ACC HWY', 'ASSAULT (IN PROGRESS): SHOTS/OUTSIDE', 'ASSAULT (IN PROGRESS): SHOTS/SCHOOL', 'ASSAULT (IN PROGRESS): SHOTS/TRANSIT', 'ASSAULT (IN PROGRESS): WEAPON/CHILD', 'ASSAULT (IN PROGRESS): WEAPON/FAMILY', 'ASSAULT (IN PROGRESS): WEAPON/INSIDE', 'ASSAULT (IN PROGRESS): WEAPON/

array(['Non Critical', 'Serious', 'Non CIP', 'Critical'], dtype=object)

In [25]:
#divide incident type into main, sub, subsub, context
df[["incident_main", "incident_sub"]] = df["typ_desc"].str.split(":", n=1, expand=True)
df[["incident_sub", "incident_subsub"]] = df["incident_sub"].str.split("/", n=1, expand=True)
df["incident_context"] = df["incident_main"].str.extract(r"\((.*?)\)")
df["incident_main"] = df["incident_main"].str.replace(r"\s*\(.*?\)", "", regex=True).str.strip()

print(df[["radio_code", "incident_main", "incident_sub", "incident_subsub","incident_context"]].head(30))

   radio_code incident_main        incident_sub incident_subsub  \
0        39H2  OTHER CRIMES          HARASSMENT         OUTSIDE   
1        39H1  OTHER CRIMES          HARASSMENT          INSIDE   
2        39Q9  OTHER CRIMES               OTHER         TRANSIT   
3        34Q6       ASSAULT               OTHER          FAMILY   
4        50P2    DISORDERLY              PERSON         OUTSIDE   
5        50G1    DISORDERLY               GROUP          INSIDE   
6        50G2    DISORDERLY               GROUP         OUTSIDE   
7        39H2  OTHER CRIMES          HARASSMENT         OUTSIDE   
8        39H1  OTHER CRIMES          HARASSMENT          INSIDE   
9        50G2    DISORDERLY               GROUP         OUTSIDE   
10       50G2    DISORDERLY               GROUP         OUTSIDE   
11       50P9    DISORDERLY              PERSON         TRANSIT   
12       34K1       ASSAULT               KNIFE          INSIDE   
13       39H1  OTHER CRIMES          HARASSMENT          INSID

In [41]:
df[df["incident_main"] == "OTHER-CRIME INCIDENT"].head(5)

,cad_evnt_id,create_date,incident_date,incident_time,nypd_pct_cd,boro_nm,patrl_boro_nm,geo_cd_x,geo_cd_y,radio_code,...,add_ts,arrivd_ts,disp_ts,closng_ts,latitude,longitude,incident_main,incident_sub,incident_subsub,incident_context
65355,100575765,2024-02-01T00:00:00.000,2024-02-01T00:00:00.000,19:15:28,69,BROOKLYN,PATROL BORO BKLYN SOUTH,1015225,170484,69M1,...,2024-02-01T19:15:28.000,2024-02-01T21:15:44.000,2024-02-01T20:04:20.000,2024-02-01T21:16:46.000,40.634570,-73.888401,OTHER-CRIME INCIDENT,MARIJUANA,INSIDE,NaN
105188,101022049,2024-02-20T00:00:00.000,2024-02-20T00:00:00.000,12:30:50,115,QUEENS,PATROL BORO QUEENS NORTH,1016054,214240,69M2,...,2024-02-20T12:30:50.000,2024-02-20T12:50:32.000,2024-02-20T12:50:30.000,2024-02-20T13:06:53.000,40.754667,-73.885208,OTHER-CRIME INCIDENT,MARIJUANA,OUTSIDE,NaN
123180,101225414,2024-02-28T00:00:00.000,2024-02-28T00:00:00.000,23:15:28,103,QUEENS,PATROL BORO QUEENS SOUTH,1041685,198146,69M1,...,2024-02-28T23:15:28.000,2024-02-29T00:20:49.000,2024-02-28T23:49:44.000,2024-02-29T00:21:14.000,40.710364,-73.792832,OTHER-CRIME INCIDENT,MARIJUANA,INSIDE,NaN
162110,101687547,2024-03-17T00:00:00.000,2024-03-17T00:00:00.000,22:01:42,10,MANHATTAN,PATROL BORO MAN SOUTH,984340,216674,69M1,...,2024-03-17T22:01:42.000,2024-03-17T22:30:47.000,2024-03-17T22:24:14.000,2024-03-17T22:32:23.000,40.761405,-73.999677,OTHER-CRIME INCIDENT,MARIJUANA,INSIDE,NaN
294338,103159581,2024-05-17T00:00:00.000,2024-05-17T00:00:00.000,12:44:04,67,BROOKLYN,PATROL BORO BKLYN SOUTH,1006206,175643,69M2,...,2024-05-17T12:44:04.000,2024-05-17T13:35:43.000,2024-05-17T12:50:33.000,2024-05-17T13:44:39.000,40.648757,-73.920879,OTHER-CRIME INCIDENT,MARIJUANA,OUTSIDE,NaN


In [43]:
df[df["incident_main"] == "OTHER CRIMES"].head(5)

,cad_evnt_id,create_date,incident_date,incident_time,nypd_pct_cd,boro_nm,patrl_boro_nm,geo_cd_x,geo_cd_y,radio_code,...,add_ts,arrivd_ts,disp_ts,closng_ts,latitude,longitude,incident_main,incident_sub,incident_subsub,incident_context
0,99844077,2024-01-01T00:00:00.000,2024-01-01T00:00:00.000,00:01:13,6,MANHATTAN,PATROL BORO MAN SOUTH,981882,209075,39H2,...,2024-01-01T00:01:13.000,2024-01-01T00:07:07.000,2024-01-01T00:04:49.000,2024-01-01T00:09:19.000,40.740547,-74.008547,OTHER CRIMES,HARASSMENT,OUTSIDE,IN PROGRESS
1,99844080,2024-01-01T00:00:00.000,2024-01-01T00:00:00.000,00:01:21,103,QUEENS,PATROL BORO QUEENS SOUTH,1042321,195155,39H1,...,2024-01-01T00:01:21.000,2024-01-01T00:24:19.000,2024-01-01T00:04:00.000,2024-01-01T00:47:24.000,40.702150,-73.790564,OTHER CRIMES,HARASSMENT,INSIDE,IN PROGRESS
2,99844104,2024-01-01T00:00:00.000,2024-01-01T00:00:00.000,00:03:04,44,BRONX,PATROL BORO BRONX,1004911,240882,39Q9,...,2024-01-01T00:03:04.000,2024-01-01T00:03:04.000,2024-01-01T00:03:04.000,2024-01-01T00:03:12.000,40.827825,-73.925345,OTHER CRIMES,OTHER,TRANSIT,IN PROGRESS
7,99844332,2024-01-01T00:00:00.000,2024-01-01T00:00:00.000,00:16:09,32,MANHATTAN,PATROL BORO MAN NORTH,999857,234502,39H2,...,2024-01-01T00:16:09.000,2024-01-01T00:45:00.000,2024-01-01T00:17:04.000,2024-01-01T00:46:12.000,40.810324,-73.943622,OTHER CRIMES,HARASSMENT,OUTSIDE,IN PROGRESS
8,99844333,2024-01-01T00:00:00.000,2024-01-01T00:00:00.000,00:16:10,108,QUEENS,PATROL BORO QUEENS NORTH,1006866,209745,39H1,...,2024-01-01T00:16:10.000,2024-01-01T00:27:12.000,2024-01-01T00:17:44.000,2024-01-01T01:11:44.000,40.742357,-73.918386,OTHER CRIMES,HARASSMENT,INSIDE,IN PROGRESS


In [48]:
df['incident_subsub'].unique()

array(['OUTSIDE', 'INSIDE', 'TRANSIT', 'FAMILY', None, 'LTD ACC HWY',
       'CHILD ABUSE', 'CHILD', 'SCHOOL', 'TRESPASS'], dtype=object)

In [26]:
# export processed dataframe to parquet (keeps original file intact)
out_path = "src/nyc_data_crime_processed.parquet"
df.to_parquet(out_path, index=False, engine="pyarrow", compression="snappy")
print(f"Exported {len(df):,} rows to {out_path}")

Exported 815,867 rows to src/nyc_data_crime_processed.parquet


In [36]:
counts_df = (
    df.groupby(["incident_main", "cip_jobs"])
      .size()
      .reset_index(name="count")
      .sort_values(["incident_main", "count"], ascending=[True, False])
)

# show results
print(counts_df.head(50))
counts_df.to_csv("incident_counts.csv", index=False)

           incident_main      cip_jobs   count
2                ASSAULT       Serious   75231
1                ASSAULT       Non CIP   27906
0                ASSAULT      Critical   19430
3               BURGLARY      Critical   20872
4               BURGLARY       Non CIP   14167
5             DISORDERLY       Non CIP  136029
7                LARCENY       Non CIP  128481
8                LARCENY       Serious   73842
6                LARCENY      Critical    2637
10          OTHER CRIMES  Non Critical  235571
9           OTHER CRIMES       Non CIP   61540
11  OTHER-CRIME INCIDENT       Non CIP      28
12               ROBBERY      Critical   12513
13               ROBBERY       Non CIP    7620
